# Code-to-Code Comparison: Dinwoodie

National Renewable Energy Laboratory\
Rob Hammond\
1 July 2024

**Note**: The results in this notebook will differ from those in prior versions quite significantly in some places due to the numerous logic changes that have both updated and improved the features provided by WOMBAT, and fixed previously unidentified bugs.

In [1]:
from copy import deepcopy
from time import perf_counter

import yaml
import pandas as pd

from wombat import Simulation
from wombat.core.library import DINWOODIE

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = "{:,.2f}".format

In [2]:
# Converting Labor values to fixed cost input for the base case
tech_salary_annual = 80000
techs = 20
capacity = 240 * 1000  # 240 -> kW
f"{tech_salary_annual * techs / capacity:.4f}"

'6.6667'

In [3]:
configs = [
    "base",
    "more_ctvs",
    "fewer_ctvs",
    "more_techs",
    "fewer_techs",
    "failure_50",
    "failure_200",
    "no_hlvs",
    "no_weather",
    "historic_weather",
    "manual_resets_only",
    "minor_repairs_only",
    "medium_repairs_only",
    "major_repairs_only",
    "major_replacements_only",
    "annual_service_only",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "fsv cost": [],
    "hlv cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
    "ctv utilization": [],
    "fsv utilization": [],
    "hlv utilization": [],
}
metrics_dict = {}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(DINWOODIE, f"{config}.yaml", random_seed=2023)
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(34)} | {(end - start) / 60:.2f} m")
    metrics_dict[config] = sim.metrics

                    dinwoodie_base | 0.35 m
               dinwoodie_more_ctvs | 0.37 m
              dinwoodie_fewer_ctvs | 0.36 m
              dinwoodie_more_techs | 0.37 m
             dinwoodie_fewer_techs | 0.37 m
              dinwoodie_failure_50 | 0.30 m
             dinwoodie_failure_200 | 0.47 m
                 dinwoodie_no_hlvs | 0.37 m
              dinwoodie_no_weather | 0.35 m
        dinwoodie_historic_weather | 0.30 m
      dinwoodie_manual_resets_only | 0.27 m
      dinwoodie_minor_repairs_only | 0.23 m
     dinwoodie_medium_repairs_only | 0.20 m
      dinwoodie_major_repairs_only | 0.19 m
 dinwoodie_major_replacements_only | 0.19 m
     dinwoodie_annual_service_only | 0.24 m


In [5]:
for config, metrics in metrics_dict.items():
    # Gather the results of interest
    years = metrics.events.year.unique().shape[0]
    mil = 1000000

    availability_time = metrics.time_based_availability(
        frequency="project", by="windfarm"
    ).values[0][0]
    availability_production = metrics.production_based_availability(
        frequency="project", by="windfarm"
    ).values[0][0]
    cf_net = metrics.capacity_factor(
        which="net", frequency="project", by="windfarm"
    ).values[0][0]
    cf_gross = metrics.capacity_factor(
        which="gross", frequency="project", by="windfarm"
    ).values[0][0]
    power_production = metrics.power_production(
        frequency="project", by="windfarm"
    ).values[0][0]
    completion_rate = metrics.task_completion_rate(
        which="both", frequency="project"
    ).values[0][0]
    parts = metrics.events[["materials_cost"]].sum().sum()
    techs = metrics.project_fixed_costs(
        frequency="project", resolution="low"
    ).operations[0]
    total = metrics.events[["total_cost"]].sum().sum()

    equipment = metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    hlv = (
        equipment[[el for el in equipment.columns if "Heavy Lift Vessel" in el]]
        .sum()
        .sum()
    )
    fsv = (
        equipment[[el for el in equipment.columns if "Field Support Vessel" in el]]
        .sum()
        .sum()
    )
    ctv = (
        equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]]
        .sum()
        .sum()
    )

    utilization = metrics.service_equipment_utilization(frequency="project")
    hlv_ur = (
        utilization[[el for el in utilization.columns if "Heavy Lift Vessel" in el]]
        .mean()
        .mean()
    )
    fsv_ur = (
        utilization[[el for el in utilization.columns if "Field Support Vessel" in el]]
        .mean()
        .mean()
    )
    ctv_ur = (
        utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]]
        .mean()
        .mean()
    )

    # Log the results of interest
    results["availability - time based"].append(availability_time)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["annual direct O&M cost"].append((total + techs) / mil / years)
    results["annual vessel cost"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["fsv cost"].append(fsv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["annual repair cost"].append(parts / mil / years)
    results["annual technician cost"].append(techs / mil / years)
    results["ctv utilization"].append(ctv_ur)
    results["fsv utilization"].append(fsv_ur)
    results["hlv utilization"].append(hlv_ur)

In [6]:
# Save the results
with open(DINWOODIE / "results" / "results_dict_v0.9.4.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(
    results.values(), columns=columns, index=results.keys()
).fillna(0)
results_df.to_csv(
    DINWOODIE / "results" / "results_data_v0.9.4.csv", index_label="result"
)

In [7]:
results_df

,base,more_ctvs,fewer_ctvs,more_techs,fewer_techs,failure_50,failure_200,no_hlvs,no_weather,historic_weather,manual_resets_only,minor_repairs_only,medium_repairs_only,major_repairs_only,major_replacements_only,annual_service_only
availability - time based,0.97,0.97,0.97,0.97,0.97,0.98,0.96,0.97,0.98,0.97,1.00,1.00,1.00,1.00,1.00,0.98
availability - production based,0.97,0.97,0.97,0.97,0.97,0.98,0.96,0.97,0.00,0.97,1.00,1.00,1.00,1.00,1.00,0.99
capacity factor - net,0.46,0.46,0.46,0.46,0.46,0.47,0.46,0.46,0.00,0.47,0.47,0.47,0.48,0.48,0.48,0.47
capacity factor - gross,0.48,0.48,0.48,0.48,0.48,0.48,0.48,0.48,0.00,0.48,0.48,0.48,0.48,0.48,0.48,0.48
power production,"9,737.45","9,741.23","9,737.45","9,737.45","9,737.45","9,814.63","9,672.96","9,728.11",0.00,"7,873.16","9,983.91","9,982.89","10,012.05","10,029.66","10,025.67","9,881.00"
task completion rate,1.00,1.00,1.00,1.00,1.00,1.00,0.99,1.00,1.00,1.00,1.00,1.00,1.00,0.91,0.95,1.00
annual direct O&M cost,16.82,19.22,16.82,17.62,16.02,10.40,21.23,5.49,15.03,16.32,3.52,3.75,3.91,3.83,9.46,4.85
annual vessel cost,10.72,10.95,10.72,10.72,10.72,6.11,13.47,1.92,9.25,10.56,1.92,1.92,1.92,2.08,6.49,1.92
ctv cost,1.92,3.20,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92
fsv cost,0.27,0.24,0.27,0.27,0.27,0.13,0.41,0.00,0.24,0.20,0.00,0.00,0.00,0.16,0.00,0.00
